# Tutorial 1c: Generate hard-links using SPACEL

In [ ]:
from SPACEL import Scube
import scanpy as sc
import numpy as np
import os
import pandas as pd
import seaborn as sns
import matplotlib
import pandas as pd
import anndata
from SPACEL import Splane
import time

In [ ]:
def load_DLPFC(root_dir='../benchmarking_data/DLPFC12', section_id='151507'):
    # 151507, ..., 151676 12 in total
    ad = sc.read_visium(path=os.path.join(root_dir, section_id), count_file=section_id+'_filtered_feature_bc_matrix.h5', load_images=True)
    ad.var_names_make_unique()

    gt_dir = os.path.join(root_dir, section_id, 'gt')
    gt_df = pd.read_csv(os.path.join(gt_dir, 'tissue_positions_list_GTs.txt'), sep=',', header=None, index_col=0)
    ad.obs['original_clusters'] = gt_df.loc[:, 6]
    keep_bcs = ad.obs.dropna().index
    ad = ad[keep_bcs].copy()
    ad.obs['original_clusters'] = ad.obs['original_clusters'].astype(int).astype(str)
    # print(ad.obs)
    return ad

In [ ]:
"""dlpfc 1"""
section_ids_list = [['151507', '151508']]
data_dir = '../../spatial_benchmarking/benchmarking_data/DLPFC12'
run_times = []
save_dir_gt = './SPACEL/align_coords'

for iter_ in range(1):
    for section_ids in section_ids_list:
        inputs = []

        i = 0
        # adata_list = []
        for section_id in section_ids:
            dataset = section_ids[0] + '_' + section_ids[1]
            
            adata = load_DLPFC(root_dir=data_dir, section_id=section_id)
            inputs.append(adata)
        
        start_time = time.time()
        if not os.path.exists(os.path.join(save_dir_gt, dataset)):
            os.makedirs(os.path.join(save_dir_gt, dataset))
        save_path = os.path.join(save_dir_gt, dataset, 'aligned_coordinates.csv')

        Scube.align(inputs,
                    cluster_key='spatial_domain',
                    n_neighbors = 15,
                    n_threads=10,
                    p=2,
                    write_loc_path=save_path
                    )        
        end_time = time.time()